In [264]:
import os
import numpy as np
from tabulate import tabulate
from tqdm import tqdm
# import keras
# import keras.backend as K

In [114]:
os.listdir()

['text',
 'distance.py',
 'nbest-sym.txt',
 'nbest.txt',
 '.ipynb_checkpoints',
 'nbest classifier.ipynb',
 'words.txt']

# read data

In [2]:
enc_phone = dict()
dec_phone = dict()

with open('words.txt') as file:
    for line in file:
        ph, num = line.strip().split(' ')
        enc_phone[ph] = num
        dec_phone[num] = ph

In [115]:
text = dict()

with open('nbest-sym.txt') as file:
    lines = file.readlines()
    change = False
    name = ''
    
    for line in lines:
        line = line.strip()
        if(line != ''):
            if(line[0] in ['F', 'M']):
                change = True
                name = line      
                text[name] = []
            elif(text[name] == []):                
                target = line.split(' ')[1:]
                text[name] = target           

In [116]:
label = dict()

with open('text') as file:
    for line in file:
        spl = line.strip().split(' ')
        label[spl[0]] = []
        for ph in spl[1:]:
            label[spl[0]].append(ph)

# Model (not use now)

In [80]:
from keras.layers import Input, Dense, Embedding, Activation, dot, Flatten
from keras.models import Model

n_lexicon = int(len(text))

def get_model():
    a = Input(shape=(25,))
    b = Input(shape=(25,))
    
    emb = Embedding(n_lexicon+1, 16)
    a_emb = emb(a)        
    b_emb = emb(b)    
                
    x = dot([a_emb, b_emb], axes=1)
    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)    
    
    model = Model(inputs=[a,b], outputs=x)
    return model

In [81]:
model = get_model()
model.compile('Adam', loss='mse', metrics=['acc'])

In [82]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
input_50 (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_48 (Embedding)        (None, 25, 16)       9600        input_49[0][0]                   
                                                                 input_50[0][0]                   
__________________________________________________________________________________________________
dot_25 (Dot)                    (None, 16, 16)       0           embedding_48[0][0]               
          

In [72]:
words_dict = []
keys = []
maxlen = 25

for key in label:
    phonemes = label[key][:maxlen]
    addition = maxlen-len(phonemes)
    if(addition > 0):
        phonemes.extend([enc_phone["<SIL>"]]*addition)    
    words_dict.append(phonemes)

#     for phone in phonemes:        
#         temp = np.zeros(n_lexicon,dtype=np.int)
#         temp[int(phone)] = 1
#         words_dict[-1].append(temp)
    
    keys.append(key)

In [73]:
words_dict = np.array(words_dict, dtype='int')
words_dict.shape

(599, 25)

In [74]:
X_train_emb_a = np.concatenate((words_dict,words_dict), axis=0)
X_train_emb_b = np.concatenate((words_dict,np.flip(words_dict,0)), axis=0)
y_train_emb = [1]*(len(X_train_emb_a)//2) + [0]*(len(X_train_emb_a)//2)
y_train_emb = np.array(y_train_emb, dtype='int')

In [75]:
X_train_emb_a.shape, X_train_emb_b.shape, y_train_emb.shape

((1198, 25), (1198, 25), (1198,))

In [76]:
model.fit(x=[X_train_emb_a, X_train_emb_b], y=y_train_emb, epochs=10, batch_size=32)

Epoch 1/10
1198/1198 [==============================] - 1s 460us/step - loss: 0.2410 - acc: 0.6202
Epoch 2/10
1198/1198 [==============================] - 0s 183us/step - loss: 0.1594 - acc: 0.8681
Epoch 3/10
1198/1198 [==============================] - 0s 185us/step - loss: 0.0623 - acc: 0.9349
Epoch 4/10
1198/1198 [==============================] - 0s 175us/step - loss: 0.0323 - acc: 0.9716
Epoch 5/10
1198/1198 [==============================] - 0s 180us/step - loss: 0.0229 - acc: 0.9733
Epoch 6/10
1198/1198 [==============================] - 0s 186us/step - loss: 0.0170 - acc: 0.9816
Epoch 7/10
1198/1198 [==============================] - 0s 175us/step - loss: 0.0137 - acc: 0.9875
Epoch 8/10
1198/1198 [==============================] - 0s 177us/step - loss: 0.0107 - acc: 0.9900
Epoch 9/10
1198/1198 [==============================] - 0s 163us/step - loss: 0.0093 - acc: 0.9917
Epoch 10/10
1198/1198 [==============================] - 0s 176us/step - loss: 0.0089 - acc: 0.9917


# edit-distance

In [291]:
#dp edit-distance 
def distance(a,b, debug=False):
    N = len(a)
    M = len(b)
    dp = np.zeros((N+1,M+1), dtype=np.int)
    
    for i in range(N+1):
        dp[i][0] = i
        
    for j in range(M+1):
        dp[0][j] = j
    
    for i in range(N):
        for j in range(M):
            best_prev = min(dp[i][j], min(dp[i+1][j], dp[i][j+1]))
            if(a[i] == b[j]):
                dp[i+1][j+1] = dp[i][j]
            else:
                dp[i+1][j+1] = best_prev + measure_weight(a[i], b[j])
    if(debug):
        return dp
    else:
        return dp[N][M]
    
    
def debug(a = list("sunday"), b = list("suundaY")):
    table = distance(a, b, debug=True)
    a = list(a)
    b = list(b)
    tabs = []
    for i, row in enumerate(table):        
        tabs.append([([''] + a)[i]])
        tabs[-1].extend(row)

    print(tabulate(tabs, headers=['']+b))
    

def isVow(x):
    return x in ['a','aa', 'i', 'ii', 'v', 'vv', 'u', 'uu', 'e', 'ee',
                'x', 'xx', 'o', 'oo', '@', '@@', 'q', 'qq', 
                'ia', 'iia', 'va', 'vva', 'ua', 'uua']
    
    
def measure_weight(x, y):
    inter_vow_weight = 3 # a -> i
    partial_inter_vow_weight = 2 # vv -> vva, v -> va
    inside_vow_weight = 1 # v -> vv, ua -> uua
    norm_weight = 1;
#     return 1
    if(x == y): return 0
    try:
        x = dec_phone[x]
        y = dec_phone[y]
    except:
        pass
    
    if(isVow(x) or isVow(y)):
        if(len(x) > len(y)):
            x, y = y, x

        if( x[0]*2 + x[1:] == y ):
            return inside_vow_weight
        elif(x + 'a' == y):
            return partial_inter_vow_weight
        else:
            return inter_vow_weight
    else:
        return norm_weight

In [274]:
def find_closest(x):    
    possible = set()
    min_dist = 1e9
    for key, val in label.items():        
        dist = distance(x, val)
        if(dist < min_dist):
            min_dist = dist            
            possible = set([" ".join(val)])
        elif(dist == min_dist):
            possible.add(" ".join(val))
    return possible, min_dist

In [215]:
keys = list(text.keys())

In [277]:
k = keys[0]
print(' '.join(text[k]))
print(' '.join(label[k]))
print(k)

possible, min_dist = find_closest(text[k])
print(possible, min_dist)

l @@ phl aa ph @@ p a j^ j uu t^
l aa k @@ j^
F5731101921_0080
{'l aa k @@ j^', 'phl ee ng^ t @@ p a j^', 'kh @@ phl ee ng^ kl a j^', 'kh aa m^ p a j^'} 8


## test accuracy

In [ ]:
%%time
acc = 0
amount = len(keys)
for key in tqdm(keys[:amount]):
    possible, min_dist = find_closest(text[key])
    ans = list(possible)[0]
    lab = " ".join(label[key])
    if(len(possible)>1):
        print(key)
        print(" ".join(text[key]))
        print(possible)
        print(lab, '\n')
    if(ans == lab): acc += 1
    
        

# print("acc {}".format(acc / amount))
acc /= amount

  3%|▎         | 15/599 [00:04<02:44,  3.54it/s]

M5730279821_0051
phl qq t^ phl ee ng^ phl ee ng^ j uu b a j^ l qq f^ j uu n^
{'l qq k^ l e n^', 'j u t^'}
p qq t^ phl ee ng^ 



  4%|▍         | 24/599 [00:06<02:32,  3.76it/s]

M5730282621_0075
phl ee ng^ phl ee ng^ phl ee ng^ j ii
{'kh aa m^ phl ee ng^ n ii', 'h aa m^ l e n^ phl ee ng^ n ii'}
l e n^ phl ee ng^ z a r a j^ k @@ d aa j^ n aa th ii 



  5%|▍         | 29/599 [00:07<02:29,  3.83it/s]

...

 50%|█████     | 301/599 [01:11<01:10,  4.21it/s]

F5731101921_0051
phl ee l p^ phl ee ng^ phl ee ng^ ch ii w a j^ l a p^ j uu t^
{'phl ee ng^ n ii khr a j^ t x ng^', 'l e n^ phl ee ng^ n ii s a m^', 'phl ee ng^ n ii khr a j^ r @@ ng^', 'j u t^'}
p qq t^ phl ee ng^ 

M5731100221_0001
k oo w aa c ii j ii k^ l e n^ phl ee ng^ ch ee ng^ ph @@ p^ j uu
{'l e n^ phl ee ng^ k @@ n^ n aa', 'l e n^ phl ee ng^ kh @@ ng^'}
k oo w aa c ii l e n^ phl ee ng^ 



 51%|█████▏    | 307/599 [01:13<01:09,  4.18it/s]

M5730282621_0031
ph qq m^ n a j^ ph e n^ l i t^ phl ee ng^ k @@ n^ n @@ n^
{'ph qq m^ n a j^ ph e n^ l i t^ phl ee ng^', 'ph qq m^ n a j^ ph e n^ l i t^ phl ee ng^ pr oo t^'}
ph qq m^ n a j^ ph e n^ l i t^ phl ee ng^ 



 52%|█████▏    | 313/599 [01:14<01:08,  4.19it/s]

In [290]:
acc

0.5659432387312187